In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import dspy
gpt3_turbo = dspy.OpenAI(model='gpt-3.5-turbo-1106', max_tokens=300)
dspy.configure(lm=gpt3_turbo)

/home/cch1006/langchain-startup/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### DSPy Example objects

In [3]:
qa_pair = dspy.Example(question="This is a question?", answer="This is an answer.")

print(qa_pair)
print(qa_pair.question)
print(qa_pair.answer)

Example({'question': 'This is a question?', 'answer': 'This is an answer.'}) (input_keys=None)
This is a question?
This is an answer.


In [4]:
trainset = [dspy.Example(report="LONG REPORT 1", summary="short summary 1"),dspy.Example(report="LONG REPORT 2", summary="short summary 2")]
print(trainset)

[Example({'report': 'LONG REPORT 1', 'summary': 'short summary 1'}) (input_keys=None), Example({'report': 'LONG REPORT 2', 'summary': 'short summary 2'}) (input_keys=None)]


#### Set up the trainset and the devset

In [7]:
# Create a larger training set
trainset = [
    dspy.Example(introduction="Tiger is a animal that is known for its strength and majestic stripes.", reason="I like tigers because they are powerful and have a regal presence."),
    dspy.Example(introduction="Lion is an animal that is often called the 'King of the Jungle' because of its imposing mane and roar.", reason="I like lions because they represent courage and leadership."),
    dspy.Example(introduction="Elephant is an animal that is the largest land mammal, known for its intelligence and long tusks.", reason="I like elephants because they are gentle giants with strong social bonds."),
    dspy.Example(introduction="Giraffe is an animal that is known for its long neck and legs, allowing it to reach leaves high in trees.", reason="I like giraffes because they are unique and graceful."),
    dspy.Example(introduction="Cheetah is an animal that is the fastest land animal, capable of reaching speeds up to 75 mph.", reason="I like cheetahs because they are incredibly swift and agile."),
    dspy.Example(introduction="Penguin is an animal that is known for its distinctive black and white plumage and ability to swim in cold waters.", reason="I like penguins because they are cute and resilient.")
]

# Create a development set
devset = [
    dspy.Example(introduction="Monkey is an animal that is known for its intelligence and ability to use tools.", reason="I like monkeys because they are playful and smart."),
    dspy.Example(introduction="Dog is an animal that is known for its loyalty and companionship to humans.", reason="I like dogs because they are faithful and bring joy to people's lives."),
    dspy.Example(introduction="Panda is an animal that is known for its distinctive black and white fur and love for bamboo.", reason="I like pandas because they are adorable and symbolize peace."),
    dspy.Example(introduction="Kangaroo is an animal that is known for its powerful hind legs and ability to hop at high speeds.", reason="I like kangaroos because they are fascinating and unique to Australia.")
]

print(trainset)
print(devset)

[Example({'introduction': 'Tiger is a animal that is known for its strength and majestic stripes.', 'reason': 'I like tigers because they are powerful and have a regal presence.'}) (input_keys=None), Example({'introduction': "Lion is an animal that is often called the 'King of the Jungle' because of its imposing mane and roar.", 'reason': 'I like lions because they represent courage and leadership.'}) (input_keys=None), Example({'introduction': 'Elephant is an animal that is the largest land mammal, known for its intelligence and long tusks.', 'reason': 'I like elephants because they are gentle giants with strong social bonds.'}) (input_keys=None), Example({'introduction': 'Giraffe is an animal that is known for its long neck and legs, allowing it to reach leaves high in trees.', 'reason': 'I like giraffes because they are unique and graceful.'}) (input_keys=None), Example({'introduction': 'Cheetah is an animal that is the fastest land animal, capable of reaching speeds up to 75 mph.',

Convert the trainset to `string` for ChainOfThought to use

In [170]:
def format_trainset(trainset):
    formatted_trainset = ""
    for i in trainset :
        formatted_trainset += "introduction: "+ i.introduction + "," + "reason: "+ i.reason + "\n "
    return (formatted_trainset)
format_trainset(trainset)

"introduction: Tiger is a animal that is known for its strength and majestic stripes.,reason: I like tigers because they are powerful and have a regal presence.\n introduction: Lion is an animal that is often called the 'King of the Jungle' because of its imposing mane and roar.,reason: I like lions because they represent courage and leadership.\n introduction: Elephant is an animal that is the largest land mammal, known for its intelligence and long tusks.,reason: I like elephants because they are gentle giants with strong social bonds.\n introduction: Giraffe is an animal that is known for its long neck and legs, allowing it to reach leaves high in trees.,reason: I like giraffes because they are unique and graceful.\n introduction: Cheetah is an animal that is the fastest land animal, capable of reaching speeds up to 75 mph.,reason: I like cheetahs because they are incredibly swift and agile.\n introduction: Penguin is an animal that is known for its distinctive black and white pluma

#### Set up the program for your DSPy predict using ChainOfThought

In [172]:
class ReasonLikesAnimal(dspy.Signature):
    """Take a look at the trainset, it is a list of examples of how you should predict the reason.Based on the introduction, sum up the features what animals have.Take a assumption of what people could say when the say the y like the animal.Start the sentence with 'I like' and end with a period."""

    trainset = dspy.InputField(desc="A list of examples of animals and the reasons why people like them.")
    introduction = dspy.InputField(desc="animal featurs and characteristics.")
    reason = dspy.OutputField(desc="The reason that I like the animal. Start the sentence with 'I like' and end with a period.")

program = dspy.ChainOfThought(ReasonLikesAnimal)
response = program(trainset = format_trainset(trainset), introduction=devset[0].introduction)
print(response)

Prediction(
    rationale='produce the reason. We can see that monkeys are highly intelligent and have the ability to problem solve and use tools, making them fascinating to observe and learn from.',
    reason='I like monkeys because they are incredibly intelligent and fascinating to watch.'
)


#### Intermediate: Using AI feedback for your metric
- the validation is based on three standards , which are reason_match, pattern_match and quality_match

In [151]:
# Define the signature for automatic assessments.
class Assess(dspy.Signature):
    """Assess the quality of a tweet along the specified dimension."""

    assessed_text = dspy.InputField()
    assessment_question = dspy.InputField()
    assessment_answer = dspy.OutputField(desc="Yes or No")

gpt4T = dspy.OpenAI(model='gpt-4-1106-preview', max_tokens=1000, model_type='chat')

def validate_answer(example, pred, trace=None):
    # introduction_match = example.introduction.lower() in pred.reason.lower()
    reason_match = example.reason.lower() in pred.reason.lower()
    pattern_match = pred.reason.lower().startswith("i like")

    quality_question = "Is the assessed text a well-written and coherent response?"
    with dspy.context(lm=gpt4T):
        quality = dspy.Predict(Assess)(assessed_text=pred.reason, assessment_question=quality_question)

    quality_match = quality.assessment_answer.lower() == 'yes'

    print(reason_match, pattern_match, quality_match)
    if trace is None:  # if we're doing evaluation or optimization
        return (pattern_match + reason_match + quality_match) / 3.0
    else:  # if we're doing bootstrapping, i.e., self-generating good demonstrations of each step
        return pattern_match + reason_match and quality_match


In [152]:
response = program(introduction=devset[0].introduction)
validate_answer(devset[0], response)

False True True


0.6666666666666666

In [132]:
print(devset[0].introduction)
print(response)
print("\n")
print(devset[0].reason)
print(response.reason)

Monkey is an animal that is known for its intelligence and ability to use tools.
Prediction(
    rationale='understand the unique features of monkeys. They have dexterous hands and are able to problem solve and communicate with each other.',
    reason='I like the monkey because of its intelligence and ability to use tools.'
)


I like monkeys because they are playful and smart.
I like the monkey because of its intelligence and ability to use tools.


#### A easy for loop to grade the devset

In [153]:
scores= []
for x in devset:
    pred = program(introduction=x.with_inputs("introduction").inputs().introduction)
    score = validate_answer(x,pred)
    scores.append(score)
print(scores)

False True True
False True True
False True True
False True True
[0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666]


#### Use dspy.evaluate to grade devset completely 

In [156]:
devset_withinputs = []
for x in devset:
    x = x.with_inputs("introduction")
    devset_withinputs.append(x)
print(devset_withinputs)
print("---------------------")

from dspy.evaluate import Evaluate

# Set up the evaluator, which can be re-used in your code.
evaluator = Evaluate(devset=devset_withinputs, num_threads=1, display_progress=True, display_table=5)

# Launch evaluation.
evaluator(program, metric=validate_answer)

[Example({'introduction': 'Monkey is an animal that is known for its intelligence and ability to use tools.', 'reason': 'I like monkeys because they are playful and smart.'}) (input_keys={'introduction'}), Example({'introduction': 'Dog is an animal that is known for its loyalty and companionship to humans.', 'reason': "I like dogs because they are faithful and bring joy to people's lives."}) (input_keys={'introduction'}), Example({'introduction': 'Panda is an animal that is known for its distinctive black and white fur and love for bamboo.', 'reason': 'I like pandas because they are adorable and symbolize peace.'}) (input_keys={'introduction'}), Example({'introduction': 'Kangaroo is an animal that is known for its powerful hind legs and ability to hop at high speeds.', 'reason': 'I like kangaroos because they are fascinating and unique to Australia.'}) (input_keys={'introduction'})]
---------------------


Average Metric: 2.6666666666666665 / 4  (66.7): 100%|██████████| 4/4 [00:00<00:00, 594.79it/s]

False True True
False True True
False True True
False True True
Average Metric: 2.6666666666666665 / 4  (66.7%)



/home/cch1006/langchain-startup/venv/lib/python3.10/site-packages/dspy/evaluate/evaluate.py:142: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)
/home/cch1006/langchain-startup/venv/lib/python3.10/site-packages/dspy/evaluate/evaluate.py:216: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0.6666666666666666' '0.6666666666666666' '0.6666666666666666'
 '0.6666666666666666']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(lambda x: f'✔️ [{x}]' if x is True else f'{x}')


,introduction,example_reason,rationale,pred_reason,validate_answer
0,Monkey is an animal that is known for its intelligence and ability to use tools.,I like monkeys because they are playful and smart.,understand the unique features of monkeys. They have dexterous hands and are able to problem solve and communicate with each other.,I like the monkey because of its intelligence and ability to use tools.,0.6666666666666666
1,Dog is an animal that is known for its loyalty and companionship to humans.,I like dogs because they are faithful and bring joy to people's lives.,"understand why people like dogs. We can start by acknowledging their loyalty and companionship, as well as their playful and loving nature.","I like dogs because they are loyal, loving, and make great companions.",0.6666666666666666
2,Panda is an animal that is known for its distinctive black and white fur and love for bamboo.,I like pandas because they are adorable and symbolize peace.,"understand why people like pandas. We can start by considering their cute and cuddly appearance, their playful nature, and their unique diet of bamboo.",I like the panda because of its adorable appearance and playful nature.,0.6666666666666666
3,Kangaroo is an animal that is known for its powerful hind legs and ability to hop at high speeds.,I like kangaroos because they are fascinating and unique to Australia.,understand why people like kangaroos. We can start by acknowledging their unique physical features and impressive agility.,I like the kangaroo for its incredible ability to hop at high speeds and its powerful hind legs.,0.6666666666666666


66.67